# FIX: Reading data into CellOracle pipeline

**Authorship:**
Adam Klie
***
**Description:**

***

# Set-up

In [1]:
# Libraries
suppressMessages(library(cicero))
suppressMessages(library(monocle3))
suppressMessages(library(rhdf5))

Warning message:
“package ‘monocle3’ was built under R version 4.2.1”
Warning message:
“package ‘Biobase’ was built under R version 4.2.3”
Warning message:
“package ‘BiocGenerics’ was built under R version 4.2.1”
Warning message:
“package ‘Biobase’ was built under R version 4.2.3”
Warning message:
“package ‘BiocGenerics’ was built under R version 4.2.1”
Warning message:
“package ‘SingleCellExperiment’ was built under R version 4.2.2”
Warning message:
“package ‘SummarizedExperiment’ was built under R version 4.2.2”
Warning message:
“package ‘MatrixGenerics’ was built under R version 4.2.1”
Warning message:
“package ‘matrixStats’ was built under R version 4.2.3”
Warning message:
“package ‘GenomicRanges’ was built under R version 4.2.3”
Warning message:
“package ‘S4Vectors’ was built under R version 4.2.3”
Warning message:
“package ‘IRanges’ was built under R version 4.2.3”
Warning message:
“package ‘GenomeInfoDb’ was built under R version 4.2.3”
Warning message:
“package ‘Gviz’ was built

In [2]:
# Paths
path_data = "/cellar/users/aklie/data/datasets/atac_v1_E18_brain_fresh_5k/analysis/celloracle/mdata.h5mu"

In [3]:
# Process mudata
indata <- H5Fopen(path_data)
indices <- indata$mod$atac$layers$counts$indices
indptr <- indata$mod$atac$layers$counts$indptr
data <- as.numeric(indata$mod$atac$layers$counts$data)
barcodes <- indata$mod$atac$obs$`_index`
peaks <- indata$mod$atac$var$`_index`
h5closeAll()

In [4]:
barcodes

NULL

In [9]:
# Build sparse matrix and binarize
indata <- Matrix::sparseMatrix(i=indices, p=indptr, x=data, index1 = FALSE)
indata@x[indata@x > 0] <- 1

In [10]:
# Format cell info
cellinfo <- data.frame(row.names=barcodes, cells=barcodes)

In [15]:
# Format peak info
peaks <- gsub(":", "-", peaks)
peakinfo <- data.frame(row.names=peaks, site_name=peaks)
peakinfo <- tidyr::separate(data = peakinfo, col = 'site_name', into = c("chr", "bp1", "bp2"), sep = "-", remove=FALSE)

In [17]:
# Add names
row.names(indata) <- row.names(peakinfo)
colnames(indata) <- row.names(cellinfo)

In [18]:
# Make CDS
input_cds <-  suppressWarnings(new_cell_data_set(indata,
cell_metadata = cellinfo,
gene_metadata = peakinfo))
input_cds <- monocle3::detect_genes(input_cds)

In [19]:
# Data preprocessing
set.seed(2017)
input_cds <- estimate_size_factors(input_cds)
input_cds <- preprocess_cds(input_cds, method = "LSI")

In [ ]:
# Dimensional reduction with umap
input_cds <- reduce_dimension(input_cds, reduction_method = 'UMAP', 
                              preprocess_method = "LSI")
umap_coords <- reducedDims(input_cds)$UMAP

In [ ]:
# Build cicero cds
cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = umap_coords)

# DONE!

---